In [3]:
# For reading, visualizing, and preprocessing data
import numpy as np
import pandas as pd
import seaborn as sns
import itertools
import time
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn import model_selection
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, matthews_corrcoef, f1_score, precision_score, recall_score, cohen_kappa_score, log_loss, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Classifiers
from sklearn.svm import NuSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
# from tensorflow.keras.models import Sequential
#from mlxtend.classifier import StackingCVClassifier 

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import Dense

### Create the Stacked LSTM model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

from keras.layers import Input, Dense, LeakyReLU, Dropout
from keras.models import Model, load_model
#from keras.callbacks.callbacks import EarlyStopping, ModelCheckpoint
# Used to ignore warnings generated from StackingCVClassifier
import warnings
warnings.simplefilter('ignore')

In [2]:
data = pd.read_csv("numeric_data.csv")
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [4]:
numeric_data = data.copy()
numeric_data.drop(columns=numeric_data.columns[0],axis=1,inplace=True)

In [7]:
X = numeric_data.iloc[:,0:93].to_numpy() 
Y = numeric_data['intrusion_no']
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.20, random_state=42)
print(X_train.shape, X_test.shape)
print(Y_train.shape, Y_test.shape)

(100778, 93) (25195, 93)
(100778,) (25195,)


In [8]:
# reshape input to be [samples, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

In [10]:
model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(93,1)))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(5))
model.compile(loss='mean_squared_error',optimizer='adam')

In [11]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 93, 50)            10400     
                                                                 
 lstm_4 (LSTM)               (None, 93, 50)            20200     
                                                                 
 lstm_5 (LSTM)               (None, 50)                20200     
                                                                 
 dense_1 (Dense)             (None, 5)                 255       
                                                                 
Total params: 51,055
Trainable params: 51,055
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=6,batch_size=50,verbose=1)

Epoch 1/6
2016/2016 [==============================] - 316s 157ms/step - loss: 0.3246 - val_loss: 0.3058
Epoch 2/6
2016/2016 [==============================] - 313s 155ms/step - loss: 0.3087 - val_loss: 0.2813
Epoch 3/6
2016/2016 [==============================] - 309s 153ms/step - loss: 0.2901 - val_loss: 0.2749
Epoch 4/6
2016/2016 [==============================] - 310s 154ms/step - loss: 0.2785 - val_loss: 0.2629
Epoch 5/6
2016/2016 [==============================] - 313s 155ms/step - loss: 0.2708 - val_loss: 0.2599
Epoch 6/6
2016/2016 [==============================] - 329s 163ms/step - loss: 0.2686 - val_loss: 0.2869


In [24]:
### Lets Do the prediction and check performance metrics
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)


788/788 [==============================] - 22s 28ms/step


In [37]:
#pred_df = pd.DataFrame(test_predict,columns=Y_test.columns)
# print("Accuracy:", model.accuracy_score(Y_test, y_pred")
# print("Recall Score - ",recall_score(Y_test,test_predict.astype('uint8'),average='micro'))
# print("F1 Score - ",f1_score(Y_test,test_predict.astype('uint8'),average='micro'))
# print("Precision Score - ",precision_score(Y_test,test_predict.astype('uint8'),average='micro'))